In [1]:
import os, re, glob, json
from os.path import join as opj
import numpy as np
import pandas as pd

import networkx as nx

import ohol_transitions as transition
import ohol_object as obj

Find object and transition files:

In [2]:
a = !pwd
baseDir = '/'.join(a[0].split('/')[0:-2])
baseDir

'/Users/nvelez/Dropbox'

Transitions:

In [3]:
gsearch = lambda *args: glob.glob(opj(*args))
data_dir = '../../OneLifeData7/'
trans_dir  = opj(data_dir, 'transitions')
trans_files = gsearch(trans_dir, '*.txt')

print(*trans_files[:5], sep='\n')

../../OneLifeData7/transitions/-1_2574.txt
../../OneLifeData7/transitions/0_702.txt
../../OneLifeData7/transitions/314_235.txt
../../OneLifeData7/transitions/2165_2165.txt
../../OneLifeData7/transitions/0_1692.txt


Objects:

In [4]:
obj_dir = opj(data_dir, 'objects')
obj_files = gsearch(obj_dir, '*txt')

print(*obj_files[:5], sep='\n')

../../OneLifeData7/objects/3644.txt
../../OneLifeData7/objects/1053.txt
../../OneLifeData7/objects/1735.txt
../../OneLifeData7/objects/3122.txt
../../OneLifeData7/objects/2228.txt


Build a dictionary of {object: mapChance} pairs

In [25]:
str_extract = lambda pattern, s: re.search(pattern, s).group(0)
int_extract = lambda pattern, s: int(str_extract(pattern, s))

obj_dict = {}
for o in obj_files:
    is_obj = re.search('nextObjectNumber|groundHeat', o) is None
    
    if is_obj:
        o_num = int_extract('[0-9]+(?=.txt)', o)

        o_data = obj.read_obj(o_num)
        obj_dict[o_num] = o_data['mapChance']

Build a dataframe of transitions:

In [27]:
tech_df = pd.DataFrame()
for i in trans_files:
    a = transition.read_transition(opj(trans_dir, i))
    tech_df = pd.concat([tech_df, pd.DataFrame.from_records(a, index=[0])], sort=True)

Helper function: Is this a transition away from a naturally occurring object?

In [ ]:
def from_natural(row):
    # pull out the original actor and original target
    # ignore special cases: -1, 0, -2 = nan
    # get the map chance for valid items
    # if ANY object has mapchance 0: (or all?)
    # return True

In [28]:
tech_df.head()

,actorMinUseFraction,autoDecaySeconds,desiredMoveDist,isTool,lastUseActor,lastUseTarget,move,newActor,newActorName,newTarget,newTargetName,noUseActor,noUseTarget,origActor,origActorName,origTarget,origTargetName,reverseUseActor,reverseUseTarget,targetMinUseFraction
0,0.0,10,1.0,False,False,False,0.0,0,Empty,2578,Cool Glass,0.0,0.0,-1,Empty,2574,Molten Glass,0.0,0.0,0.0
0,0.0,0,1.0,False,False,False,0.0,425,Wolf Skin,695,Wolf Crown,NaN,NaN,0,Empty,702,Wolf Crown with Wolf Skin,0.0,0.0,0.0
0,NaN,0,NaN,True,False,False,NaN,0,Empty,317,Crucible with Iron,NaN,NaN,314,Wrought Iron,235,Clay Bowl - empty,NaN,NaN,NaN
0,0.0,0,1.0,True,False,False,0.0,235,Clay Bowl - empty,3699,Bowl with Raw Rubber Balls,0.0,0.0,2165,Bowl with Raw Rubber Ball,2165,Bowl with Raw Rubber Ball,0.0,1.0,0.0
0,0.0,0,1.0,False,False,False,0.0,1719,Hungry Schnauser Puppy - held,1706,Schnauser with Puppies -2,0.0,0.0,0,Empty,1692,Schnauser with Puppies -3,0.0,0.0,0.0


In [29]:
tech_df.query('origActor == 314')

,actorMinUseFraction,autoDecaySeconds,desiredMoveDist,isTool,lastUseActor,lastUseTarget,move,newActor,newActorName,newTarget,newTargetName,noUseActor,noUseTarget,origActor,origActorName,origTarget,origTargetName,reverseUseActor,reverseUseTarget,targetMinUseFraction
0,NaN,0,NaN,True,False,False,NaN,0,Empty,317,Crucible with Iron,NaN,NaN,314,Wrought Iron,235,Clay Bowl - empty,NaN,NaN,NaN
0,NaN,0,NaN,True,False,False,NaN,0,Empty,316,Crucible with Iron and Charcoal,NaN,NaN,314,Wrought Iron,318,Crucible with Charcoal,NaN,NaN,NaN


In [ ]:
col = ['origActor','origTarget','newActor','newTarget']
tech_tree_df = tech_df[col]
tech_tree_df = tech_tree_df.query('(origActor >= 0) and (origTarget >= 0)').reset_index(drop = True)
tech_tree_df.head()

In [33]:
transition.read_transition(opj(trans_dir, '-1_52.txt'))

{'origActor': -1,
 'origTarget': 52,
 'newActor': 0,
 'newTarget': 50,
 'autoDecaySeconds': 20,
 'lastUseActor': False,
 'lastUseTarget': False,
 'origActorName': 'Empty',
 'origTargetName': 'Fruiting Milkweed',
 'newActorName': 'Empty',
 'newTargetName': 'Milkweed',
 'isTool': False}

In [ ]:
def read_obj_file(obj_id):
    obj_str = str(obj_id)
    ## Read object file\n",
    obj_file = baseDir + '/OneLifeData7/objects/%s.txt' % obj_str
    with open(obj_file, 'r') as file:
        raw_data = file.read().splitlines()

    return raw_data

In [ ]:
#determine if this is a natural object
nat_obj = []
for i in range(1,len(tech_tree_df)):
    try:
        b = read_obj_file(i)[7].split('#')[0]
        if float(b.split('=')[1]) != 0:
            nat_obj.append(i)
    except:
        FileNotFoundError

In [ ]:
max_item_id = max(np.max(tech_tree_df).to_list())
#replace negative values
tech_tree_df = tech_tree_df.replace(-1, max_item_id+1)
#make a list of all objects
items = np.arange(max_item_id+2)
print(max_item_id)

In [ ]:
#BruteForce Recursion. Very slow

# changed = True
# while changed:
#     changed = False
#     #loop over all items  
#     for item in items:
        
#         #loop over the dataframe
#         for i in range(0,len(tech_tree_df)):
#             item1 = tech_tree_df.iloc[i]['origActor']
#             item2 = tech_tree_df.iloc[i]['origTarget']
#             product1 = tech_tree_df.iloc[i]['newActor']
#             product2 = tech_tree_df.iloc[i]['newTarget']
            
#             if (item == product1) or (item == product2):
#                 #get out of while loop is the depth of the item does not change
#                 changed = (depth[item1]+depth[item2]+1) < depth[item]
#                 #update the depth of this item
#                 depth[item] = min(depth[item], depth[item1]+depth[item2]+1)

In [ ]:
#creating a nested dictionary that stores what item plus what other item lead to what products
d = dict()
for item in items:
    d[item] = {}
    recipy = tech_tree_df.query('(origActor == @item) or (origTarget == @item)')
    if len(recipy):
        for j in range(0,len(recipy)):
            ingredients = recipy.iloc[j].tolist()[0:2]
            ingredient = np.delete(ingredients, np.where(ingredients == item)[0][0])
            products = recipy.iloc[j].tolist()[2:4]
            d[item][ingredient[0]] = products

In [ ]:
#initialize depth
depth = {}
for item in items:
    if item in nat_obj:
        depth[item] = 0
    else:
        depth[item] = 1000
depth[0] = 0 #empty hand
depth[len(depth)-1] = 0 #empty ground

In [ ]:
#create a root to start from
root = np.transpose(np.append(0,nat_obj))

while len(root):
    #take an item from root and remove it
    item = root[0]
    root = np.delete(root,0)
    print(root)
    #put all products associated with this item into root
    for key in d[item]: #looping through other ingredients
        for product in d[item][key]:
            if depth[product] > (depth[item] + depth[key] + 1):
                depth[product] = depth[item] + depth[key] + 1
                root = np.append(root, product)
                

In [ ]:
depth.values()